In [1]:
import pandas as pd
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
# read in and replace nulls
tweets = pd.read_csv('csv1.csv', delimiter=',')
tweets = tweets.replace(np.nan, '', regex=True)

In [4]:
analyser = SentimentIntensityAnalyzer()

def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    neg = score['neg']
    pos = score['pos']
    neu = score['neu']
    compound= score['compound']
    
    return [neg,pos,neu,compound, sentence]

In [16]:
#https://medium.com/analytics-vidhya/simplifying-social-media-sentiment-analysis-using-vader-in-python-f9e6ec6fc52f
text = tweets['TEXT'].tolist()
sentiments = [sentiment_analyzer_scores(s) for s in text]

In [22]:
sentiments_df = pd.DataFrame(sentiments, columns = ['NEGATIVE_SCORE', 'POSITIVE_SCORE', 'NEUTRAL_SCORE', 'COMPOUND', 'SENTENCE'])

In [44]:
finalFrame = tweets.join(sentiments_df)
finalFrame = finalFrame.iloc[:,1:-1]

In [67]:
# hashtags for each record
hashtags = tweets['HASHTAGS'].tolist()
sepHash = [i.split() for i in hashtags]

In [74]:
#[i for i in sepHash if len(i) > 0 ]
sepHash

[[],
 [],
 [],
 [],
 ['#CX', '#COVID', '#custserv'],
 ['#principalsadvocate'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['#coronavirus', '#MaskUpWyo'],
 [],
 [],
 ['#MaskUpWyoming', '#Covid_19', '#masksprotectusall'],
 [],
 ['#Covid_19', '#maskUpWyoming', '#protectothers'],
 [],
 [],
 [],
 [],
 [],
 ['#billyjean', '#MichaelJackson', '#listenagain'],
 [],
 ['#VAntagePoint'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['#WearAMask'],
 [],
 [],
 [],
 ['#COVID'],
 [],
 ['#Wyoming', '#COVID', '#WYsThat'],
 [],
 ['#VAntagePoint'],
 [],
 [],
 [],
 [],
 ['#coronavirus', '#TheShowsMustGoOn'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['#gofundme',
  '#disabled',
  '#COVID',
  '#help',
  '#struggling',
  '#struggle',
  '#veteran'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['#libm409', '#librarydesign'],
 [],
 [],
 [],
 [],
 [],
 ['#coronavirus', '#MaskUpWyo'],
 ['#A